# Assignment 6

### Part A

In [2]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [3]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [4]:
def exp_return(data, P):
    data['expected with probability'] = data['Price in 1 Year'] * data['Probability']

    return sum(data['expected with probability']) / P - 1

exp_return(data, 40)

0.07499999999999996

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [5]:
def volatility(data, P):
    expected_return = exp_return(data, P)

    data['var_temp'] = data['Probability'] * ((data['Price in 1 Year'] / P - (1 + expected_return)) ** 2)

    var = sum(data['var_temp'])
    sd = var ** 0.5

    return sd

print(volatility(data, 40))

0.21301408404140795


Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [6]:
def interval(data, P):
    z = 1.96
    n = len(data)
    expected_return = exp_return(data, P)
    stdev = volatility(data, P)
    stderr = stdev / (n ** 0.5)

    return [expected_return - z * stderr, expected_return + z * stderr]

print(interval(data, 40))

[-0.1660481279744774, 0.31604812797447734]


Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [7]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    return rf + beta * (market_return - rf)

print(exp_return_capm(-0.6,0.1, 0.02))

-0.028


### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [14]:
def clean_data(path):
    data = pd.read_csv(path)

    data = data.dropna()
    data = data[(data['RET'] != 'B') & (data['RET'] != 'C')]

    groups = data.groupby(["CUSIP"], group_keys = False)
    filtered = groups.filter(lambda x: len(x) >= 500).reset_index().drop(['index'], axis = 1)
    
    return filtered

clean_data('monthlycrsp_mod6.csv')


PERMNO        date     CUSIP        RET
0        10065  1970-01-30  00621210  -0.144375
1        10065  1970-02-27  00621210   0.105882
2        10065  1970-03-31  00621210   0.017857
3        10065  1970-04-30  00621210  -0.078947
4        10065  1970-05-29  00621210  -0.020952
...        ...         ...       ...        ...
403311   84129  2022-08-31  98970110   0.016315
403312   84129  2022-09-30  98970110  -0.075777
403313   84129  2022-10-31  98970110   0.021235
403314   84129  2022-11-30  98970110   0.005583
403315   84129  2022-12-30  98970110  -0.051332

[403316 rows x 4 columns]

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [107]:
def avg_return(data):
    # Write your code here
    
    data['RET'] = data['RET'].astype(float)

    dict = {
        "CUSIP" : [],
        "Average Return" : []
    }

    grouped = data.groupby(['CUSIP'], group_keys = False)

    dict["CUSIP"] = [name for name, group in grouped]
    dict["Average Return"] = [group['RET'].mean() for name, group in grouped]

    return pd.DataFrame(data = dict)

ret_data = avg_return(clean_data('monthlycrsp_mod6.csv'))

In [122]:
ret_data.head()

CUSIP  Average Return
0  (00036110,)        0.012326
1  (00095710,)        0.011339
2  (00105510,)        0.018398
3  (00108M10,)        0.011101
4  (00120410,)        0.012135

In [138]:
def security_volatilities(data):
    # Write your code here

    data['RET'] = data['RET'].astype(float)

    dict = {
        "CUSIP" : [],
        "Volatility" : []
    }

    avgret = avg_return(data)

    groups = data.groupby(["CUSIP"], group_keys=False)

    for name, group in groups:
        var = 0
        for i in range(len(group)):
            var += (group.iloc[i]['RET'] - avgret[avgret['CUSIP'] == name]['Average Return'].values[0]) ** 2

        var = var/(len(group) - 1)
        stdev = var ** 0.5

        dict["CUSIP"].append(name)
        dict["Volatility"].append(stdev)

    return pd.DataFrame(data = dict)

vol_data = security_volatilities(clean_data('monthlycrsp_mod6.csv'))

In [127]:
vol_data.head()

CUSIP  Volatility
0  (00036110,)    0.122475
1  (00095710,)    0.089881
2  (00105510,)    0.099828
3  (00108M10,)    0.171732
4  (00120410,)    0.051050

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [139]:
def correlation(ret_data, vol_data):
    dict = {
        "Average Return" : ret_data["Average Return"],
        "Volatility" : vol_data["Volatility"]
    }

    df = pd.DataFrame(dict).corr()

    return df.iloc[1].iloc[0]

print(correlation(ret_data, vol_data))

0.4483709072705961
